# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu)

**Description** Demo for Intercomparison Script: extracts test points, provided from hand labeled maps, from industry covermaps (Harvest, Copernicus, ESA, GLAD) and determines accuracy across maps.


In [26]:
# Authenticate notebook for Earth Engine use. Only needs to be run once, but is required for functions to work!
# !earthengine authenticate

In [27]:
import sys
sys.path.append('../')
from src.compare_covermaps import *

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas
import ee
from shapely import wkt

In [41]:
# set1_maps = [TARGETS["harvest_togo"], TARGETS["harvest_kenya"], TARGETS["harvest_tanzania"], TARGETS["copernicus"], TARGETS["esa"], TARGETS["glad"]]
set1_maps = [TARGETS["harvest_togo"], TARGETS["harvest_kenya"], TARGETS["harvest_tanzania"], TARGETS["gfsad-gcep"], TARGETS["gfsad-lgrip"], TARGETS["gfsad"]]
set1_countries = ["Togo", "Kenya", "Tanzania"]

In [42]:
test1 = TestCovermaps(test_countries=set1_countries, covermaps=set1_maps)

In [43]:
# test1_test = test1.get_test_points()
test1_test = pd.read_csv("../../intercomparison-data/extracted/test/tgo_tan_ken_2019_clipped.csv").drop("Unnamed: 0", axis=1)
test1_test["geometry"] = test1_test["geometry"].apply(wkt.loads)
test1_test = gpd.GeoDataFrame(test1_test, crs="epsg:4326")

In [44]:
extracted = test1.extract_covermaps(test1_test)

[Togo] sampling harvest_togo...
[Togo] sampling gfsad-gcep...
Extracting error: length of sampled dataset is not the same as testing dataset
[Togo] sampling gfsad-lgrip...
Extracting error: length of sampled dataset is not the same as testing dataset
[Togo] sampling gfsad...
[Kenya] sampling harvest_kenya...
[Kenya] sampling gfsad-gcep...
[Kenya] sampling gfsad-lgrip...
[Kenya] sampling gfsad...
[Tanzania] sampling harvest_tanzania...
[Tanzania] sampling gfsad-gcep...
[Tanzania] sampling gfsad-lgrip...
[Tanzania] sampling gfsad...


In [45]:
a=test1.evaluate()
test1.results["Tanzania"]

evaluating maps...
dataset: harvest_togo | country: Togo
dataset: gfsad-gcep | country: Togo
dataset: gfsad-lgrip | country: Togo
dataset: gfsad | country: Togo
dataset: harvest_kenya | country: Kenya
dataset: gfsad-gcep | country: Kenya
dataset: gfsad-lgrip | country: Kenya
dataset: gfsad | country: Kenya
dataset: harvest_tanzania | country: Tanzania
dataset: gfsad-gcep | country: Tanzania
dataset: gfsad-lgrip | country: Tanzania
dataset: gfsad | country: Tanzania


,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest_tanzania,Tanzania,0.708861,0.760573,767,770,0.584094,0.936364,0.901408,0.693269
0,gfsad-gcep,Tanzania,0.763359,0.798308,767,770,0.651890,0.944156,0.920810,0.731388
0,gfsad-lgrip,Tanzania,0.771103,0.804164,767,770,0.661017,0.946753,0.925182,0.737108
0,gfsad,Tanzania,0.484596,0.597267,767,770,0.379400,0.814286,0.670507,0.568450


In [46]:
test1.results["Togo"]

,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest_togo,Togo,0.821346,0.820930,215,215,0.823256,0.818605,0.819444,0.822430
0,gfsad-gcep,Togo,0.471642,0.588372,215,215,0.367442,0.809302,0.658333,0.561290
0,gfsad-lgrip,Togo,0.448485,0.576744,215,215,0.344186,0.809302,0.643478,0.552381
0,gfsad,Togo,0.336134,0.448837,215,215,0.279070,0.618605,0.422535,0.461806


In [47]:
test1.results["Kenya"]

,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest_kenya,Kenya,0.921514,0.888743,571,193,0.873905,0.932642,0.974609,0.714286
0,gfsad-gcep,Kenya,0.960270,0.938482,571,193,0.994746,0.772021,0.928105,0.980263
0,gfsad-lgrip,Kenya,0.959391,0.937173,571,193,0.992995,0.772021,0.927987,0.973856
0,gfsad,Kenya,0.839709,0.769634,571,193,0.807356,0.658031,0.874763,0.535865


### ====== Testing Zone =======

In [57]:
dyn = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").select("crops").filterDate("2019-01-01", "2019-12-31")

In [59]:
test_coll = ee.FeatureCollection(test1_test.apply(lambda x: create_point(x), axis=1).to_list())
sampled = extract_points(ic=dyn, fc=test_coll, resolution=10)

Exception: Computation timed out.

In [ ]:
sampled